In [1]:
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
from nltk.corpus import stopwords
import re as re
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
#from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.clustering import LDA, LDAModel
import nltk
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import size
from pyspark.sql.functions import udf, col
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, struct
import pyspark.sql.types as T 
import string
import nltk
from pyspark.sql import Row

import os
import sys
from pyspark.sql import SparkSession



In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark =SparkSession.builder.master("local[1]").appName('SparkProject').getOrCreate() 

In [3]:
#data  = spark.read.json(r"C:\Users\EndUser\Desktop\Data\AbhishekData\tweets.json")
#df = spark.read.json("tweets2.csv")
df = spark.read.json(r"C:\Users\EndUser\Desktop\Data\AbhishekData\tweets90.json")

In [4]:
df.head()

Row(content='.@VW will introduce heads-up display with augmented-reality in its upcoming #EVs. https://t.co/CINqosgiCD', date='2017-01-31T23:49:34+00:00', latitude=None, longitude=None, tweetid=826578200402616320, userid=1187313746, userlocation='')

In [5]:
df = df.dropDuplicates(['tweetid'])
df = df.na.drop()
df = df.filter("content != ''")

In [6]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_number(tweet):
    tweet = re.sub('([0-9]+)', '', tweet) 
    return tweet

def remove_users(tweet):
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    return tweet

In [7]:
from pyspark.sql.types import StringType

remove_emoji=udf(remove_emoji)
remove_users=udf(remove_users)
remove_number=udf(remove_number)

df=df.withColumn('content', remove_emoji(df['content']))
df=df.withColumn('content', remove_users(df['content']))
df=df.withColumn('content', remove_number(df['content']))

In [8]:

df = df.withColumn('content', F.regexp_replace('content', r'http\S+', ''))
df = df.withColumn('content', F.regexp_replace('content', '@\w+', ''))
df = df.withColumn('content', F.regexp_replace('content', '#', ''))
df = df.withColumn('content', F.regexp_replace('content', 'RT', ''))
df = df.withColumn('content', F.regexp_replace('content', ':', ''))
df = df.withColumn('content', F.regexp_replace('content', '\n', ''))
df = df.withColumn('content', F.regexp_replace('content', '|', ''))
df = df.withColumn('content', F.regexp_replace('content', '[^\sa-zA-Z0-9]', ''))
df = df.withColumn("content",F.lower('content'))
df = df.withColumn("content",F.rtrim('content'))

In [9]:
df.head()

Row(content='evs  continued to grow rapidly  in  while lithiumion battery prices decline  automobiles renewables', date='2017-01-01T16:03:50+00:00', latitude=18.845343, longitude=72.74484, tweetid=815589355829854208, userid=14158133, userlocation='mumbai')